#API returning values for the last 12 months and n months predicted inflation in Poland

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import load_model
import joblib

!pip install flask-ngrok
from flask import Flask
from flask_ngrok import run_with_ngrok

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
table = pd.read_html("https://stat.gov.pl/obszary-tematyczne/ceny-handel/wskazniki-cen/wskazniki-cen-towarow-i-uslug-konsumpcyjnych-pot-inflacja-/miesieczne-wskazniki-cen-towarow-i-uslug-konsumpcyjnych-od-1982-roku/")

In [3]:
table = table[0].iloc[81:122, 1:14]
months = [str(x) for x in range(1, 13)]
table = table.set_axis(['year']+months, axis=1)
table.iloc[23, 0] = '1999'
table = table.set_index('year')

In [4]:
df = table.astype(float)
df = (df - 1000)/10
df = df.iloc[::-1]

In [5]:
df = pd.DataFrame(df.stack()).iloc[-24:]
new_dates = pd.date_range(end='-'.join(df.index[-1]), periods=24, freq='MS')
df.index = new_dates
df.columns = ['True_value']

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/Flask/EconomicIndicators

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/Flask/EconomicIndicators


In [7]:
model = load_model("simpleRNN.h5")
scaler = joblib.load("RNNscaler.pkl")

In [8]:
def return_predictions(model, scaler, periods):

    forecast = []
    data = scaler.transform(df.copy())

    for i in range(periods['months_number']):
      eval_batch = data[-24:]
      current_batch = eval_batch.reshape((1, 24, 1))   
      current_pred = model.predict(current_batch)[0]
      forecast.append(current_pred) 
      data = np.append(data, current_pred[0])
    forecast = scaler.inverse_transform(forecast)
    forecast_index = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=periods['months_number'], freq='MS')
    forecast_df = pd.DataFrame(data=forecast, index=forecast_index, columns=['Forecast'])
    return forecast_df.transpose().to_json(orient='columns', date_format='iso', date_unit='s')

#Flask app

In [9]:
from flask import Flask, request, jsonify

app = Flask(__name__)
app.config['SECRET_KEY'] = 'ahggsady65gbsjja'
run_with_ngrok(app)

@app.route('/', methods=['POST'])
def home():
  last12 = df.iloc[-12:].transpose().to_json(orient='columns', date_format='iso', date_unit='s')

  content = request.json
    
  predicted = return_predictions(model, scaler, content)
  
  #return json.loads(last12), json.loads(predicted)
  return jsonify({'last12': last12, 'predicted': predicted})

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://88d3-34-148-255-77.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 16:02:04] "POST / HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 16:02:27] "POST / HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 16:03:05] "POST / HTTP/1.1" 200 -
